# Basic concepts (very important)

### Entity path

```text
"world/points"
"camera/image"
"camera/rays"
```

Like ROS topics, but hierarchical.

### Time

You explicitly set time:

```python
rr.set_time("video_frame", sequence=frame_idx)
```

or

```cpp
rr::set_time("video_frame", frame_idx);
```

Without time, everything is static.

---

# Hello World (Python & C++)

## Python

```python
import rerun as rr

rr.init("rerun_demo", spawn=True)

rr.log("text", rr.TextLog("Hello Rerun"))
```

## C++

```cpp
#include <rerun.hpp>

int main() {
    rerun::init("rerun_demo", /*spawn=*/true);
    rerun::log("text", rerun::TextLog("Hello Rerun"));
}
```

---

# Logging Images

## Python

```python
import rerun as rr
import numpy as np
import cv2

rr.init("image_demo", spawn=True)

img = cv2.imread("image.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

rr.log("camera/image", rr.Image(img))
```

## C++

```cpp
#include <rerun.hpp>
#include <opencv2/opencv.hpp>

cv::Mat img = cv::imread("image.png");
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

rerun::log(
    "camera/image",
    rerun::Image(img.data, {img.cols, img.rows}, rerun::ColorModel::RGB)
);
```

---

# 6. Video (image sequence over time)

### Key idea:

**Video = images + time**

## Python

```python
cap = cv2.VideoCapture("video.mp4")

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rr.set_time_sequence("frame", frame_idx)
    rr.log("camera/image", rr.Image(frame))

    frame_idx += 1
```

## C++

```cpp
cv::VideoCapture cap("video.mp4");
int frame_idx = 0;

cv::Mat frame;
while (cap.read(frame)) {
    cv::cvtColor(frame, frame, cv::COLOR_BGR2RGB);

    rerun::set_time_sequence("frame", frame_idx);

    rerun::log(
        "camera/image",
        rerun::Image(frame.data, {frame.cols, frame.rows})
    );

    frame_idx++;
}
```

---

# 7. 2D & 3D Points

## 2D keypoints (Python)

```python
pts = np.array([[100, 50], [120, 80], [200, 90]])

rr.log(
    "image/keypoints",
    rr.Points2D(pts, radii=3)
)
```

## 3D point cloud (Python)

```python
points = np.random.randn(1000, 3)

rr.log(
    "world/points",
    rr.Points3D(points)
)
```

## C++

```cpp
std::vector<rerun::Vec3D> points;
for (int i = 0; i < 1000; ++i) {
    points.emplace_back(randf(), randf(), randf());
}

rerun::log("world/points", rerun::Points3D(points));
```

---

# 8. Shapes (Lines, Boxes, Arrows)

## Line strip (trajectory)

### Python

```python
trajectory = np.cumsum(np.random.randn(100, 3) * 0.1, axis=0)

rr.log(
    "world/trajectory",
    rr.LineStrips3D([trajectory])
)
```

### C++

```cpp
std::vector<rerun::Vec3D> traj;
...
rerun::log(
    "world/trajectory",
    rerun::LineStrips3D({traj})
);
```

---

## Boxes

```python
rr.log(
    "world/box",
    rr.Boxes3D(
        centers=[[0,0,0]],
        sizes=[[1,1,1]]
    )
)
```

---

# 9. Camera (THIS IS CRUCIAL)

Rerun treats cameras **explicitly** — perfect for SLAM.

---

## 9.1 Camera intrinsics

### Python

```python
rr.log(
    "camera",
    rr.Pinhole(
        resolution=[640, 480],
        focal_length=[500, 500],
        principal_point=[320, 240]
    )
)
```

### C++

```cpp
rerun::log(
    "camera",
    rerun::Pinhole(
        {640, 480},
        {500.0f, 500.0f},
        {320.0f, 240.0f}
    )
);
```

---

## 9.2 Camera pose (extrinsics)

Camera pose = transform in world.

### Python

```python
rr.log(
    "camera",
    rr.Transform3D(
        translation=[0, 0, 1],
        rotation=rr.Quaternion(xyzw=[0,0,0,1])
    )
)
```

### C++

```cpp
rerun::log(
    "camera",
    rerun::Transform3D(
        rerun::Vec3D{0,0,1},
        rerun::Quaternion{0,0,0,1}
    )
);
```

---

# 10. Camera rays (what you explicitly asked)

Camera rays = **Lines3D from camera center through pixels**

---

## Python: Rays through selected pixels

```python
camera_origin = np.array([0, 0, 0])

pixels = np.array([
    [320, 240],
    [400, 240],
    [320, 300],
])

directions = np.array([
    [0, 0, 1],
    [0.1, 0, 1],
    [0, 0.1, 1],
])

directions /= np.linalg.norm(directions, axis=1, keepdims=True)

rays = []
for d in directions:
    rays.append([camera_origin, camera_origin + d * 5])

rr.log(
    "camera/rays",
    rr.LineStrips3D(rays)
)
```

---

## C++: Camera rays

```cpp
std::vector<std::vector<rerun::Vec3D>> rays;

rerun::Vec3D origin{0,0,0};
rerun::Vec3D dir{0.1f, 0.0f, 1.0f};

rays.push_back({origin, origin + dir * 5.0f});

rerun::log("camera/rays", rerun::LineStrips3D(rays));
```

---

# 11. Hover → ray interaction (your memory)

What you remembered:

* **hovering pixel → ray**
* requires:

  * image
  * pinhole
  * transform
  * same entity path

If you log **image + pinhole + transform** under `"camera"`:

```text
camera/image
camera (Pinhole)
camera (Transform3D)
```

Rerun automatically enables:

* pixel hover
* ray visualization
* projection into 3D

This is **not a toggle** — it’s **data-driven**.

---

# 12. Recommended structure for SLAM / VO

```text
world/
 ├─ points
 ├─ trajectory
camera/
 ├─ image
 ├─ keypoints
 ├─ rays
 ├─ (Pinhole)
 ├─ (Transform3D)
```

Time axis:

```text
frame = k
```

---

# 13. Common mistakes (very common)

❌ Logging image without pinhole
❌ Logging pose under different entity path
❌ Forgetting to set time
❌ Mixing camera frame vs world frame

---

# 14. Minimal SLAM-style loop (Python)

```python
rr.init("slam", spawn=True)

for k in range(100):
    rr.set_time_sequence("frame", k)

    rr.log("camera", rr.Transform3D(translation=[0, 0, k*0.1]))
    rr.log("camera/image", rr.Image(image))
    rr.log("world/points", rr.Points3D(points[k]))
```

---

# 15. What I can do next for you

If you want, I can:

✅ build a **full VO / SLAM Rerun template**
✅ show **projecting 3D points into image**
✅ show **feature tracks across frames**
✅ integrate **OpenCV + Eigen + Sophus**
✅ show **IMU + camera frames**
✅ replicate **KITTI-style visualization**

Just tell me which one you want next.
